In [1]:
#connect to drive for getting dataset
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!unzip '/content/drive/My Drive/spectrograms.zip' #unzip the spectrograms file which contain the images of spectrograms

Streaming output truncated to the last 5000 lines.
  inflating: spectrograms/54-28-38--.png  
  inflating: spectrograms/54-29-39--.png  
  inflating: spectrograms/54-30-40--.png  
  inflating: spectrograms/54-3-13--.png  
  inflating: spectrograms/54-31-41--.png  
  inflating: spectrograms/54-32-42--.png  
  inflating: spectrograms/54-33-43--.png  
  inflating: spectrograms/54-34-44--.png  
  inflating: spectrograms/5-43-53--.png  
  inflating: spectrograms/54-35-45--.png  
  inflating: spectrograms/54-36-46--.png  
  inflating: spectrograms/54-37-47--.png  
  inflating: spectrograms/54-38-48--.png  
  inflating: spectrograms/54-39-49--.png  
  inflating: spectrograms/54-40-50--.png  
  inflating: spectrograms/54-4-14--.png  
  inflating: spectrograms/54-41-51--.png  
  inflating: spectrograms/54-42-52--.png  
  inflating: spectrograms/54-43-53--.png  
  inflating: spectrograms/54-44-54--.png  
  inflating: spectrograms/5-44-54--.png  
  inflating: spectrograms/54-45-55--.png  
  infla

#codes start here

In [3]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [4]:
!pip install pydub

In [5]:
import pandas as pd
import numpy as np
from keras.models import Model
from keras.layers import Input
from keras.layers import Activation
from keras.layers import Conv2D
from keras.layers import MaxPooling2D,Bidirectional,LSTM,Reshape,CuDNNLSTM,BatchNormalization,Flatten,Dropout,Dense
from keras.layers import add
from keras.utils import plot_model

from keras.models import Model
import copy
import warnings
warnings.filterwarnings('ignore')
import cv2
import keras
from keras import backend as K
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, BatchNormalization, Flatten, Input
from keras.layers import Conv2D, Activation, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img, img_to_array
from keras.applications.resnet50 import preprocess_input, ResNet50
import matplotlib
import matplotlib.pylab as plt
import numpy as np
import seaborn as sns
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import keras

import matplotlib
matplotlib.use('Agg') # No pictures displayed

import matplotlib.pyplot as plt
import librosa
import os
import matplotlib
import pylab
import librosa
import librosa.display
import numpy as np
from pydub import AudioSegment
from pydub import AudioSegment
from pydub.utils import make_chunks
#filler
from glob import glob
import cv2
import os
from keras import optimizers

Using TensorFlow backend.


In [6]:
data = pd.read_csv('/content/drive/My Drive/labeling.csv')
data.head()

,filename,filler,repetition
0,"0-0-10-,f,f,f-.png",1,0
1,"0-1-11-,f,f,f-.png",1,0
2,0-10-20--.png,0,0
3,"0-100-110-,f-.png",1,0
4,"0-101-111-,f-.png",1,0


In [7]:
files = []
labels = []
for i in np.array(data):
  files.append(i[0])
  labels.append([i[1:]])

In [8]:
np.array(files).shape,np.array(labels).shape

((12257,), (12257, 1, 2))

In [9]:
def resnet_model_dilation(size = (256,2048,1)):
    ''' This model is build using keras module from the paper https://arxiv.org/pdf/1910.12590.pdf
    inputs are to be resized of 256,2048,1  and the no of classification items. I have fixed to binary as default
    output is the model
    '''
    input  = Input(shape = size)
    bnEps=2e-5
    bnMom=0.9


    c1 = Conv2D(64, (7,7), padding='same',strides=2,activation='relu', use_bias=False,kernel_initializer='glorot_uniform')(input)
    b1 = BatchNormalization(epsilon=bnEps, momentum=bnMom)(c1)

    c2 = conv1 = Conv2D(32, (3,3),strides=2, padding='same', use_bias=False,kernel_initializer='glorot_uniform')(input)
    b2 = BatchNormalization(epsilon=bnEps, momentum=bnMom)(c2)
    a2 = Activation('relu')(b2)

    c3 = conv1 = Conv2D(64, (3,3), padding='same', use_bias=False,kernel_initializer='glorot_uniform')(a2)
    b3 = BatchNormalization(epsilon=bnEps, momentum=bnMom)(c3)
    a3 = Activation('relu')(b3)

    c4 = conv1 = Conv2D(64, (3,3), padding='same', use_bias=False,kernel_initializer='glorot_uniform')(a3)
    b4 = BatchNormalization(epsilon=bnEps, momentum=bnMom)(c4)

    m1  = add([c1, b4])
    a4 = Activation('relu')(m1)


    #-----------------------------------------------layer 2----------------------------------------------------------------------------

    c1 = Conv2D(128, (3,3),strides=2, padding='same',activation='relu', use_bias=False,kernel_initializer='glorot_uniform')(a4)
    b1 = BatchNormalization(epsilon=bnEps, momentum=bnMom)(c1)

    c2 = conv1 = Conv2D(64, (3,3),strides=2, padding='same', use_bias=False,kernel_initializer='glorot_uniform')(a4)
    b2 = BatchNormalization(epsilon=bnEps, momentum=bnMom)(c2)
    a2 = Activation('relu')(b2)

    c3 = conv1 = Conv2D(128, (3,3), padding='same', use_bias=False,kernel_initializer='glorot_uniform')(a2)
    b3 = BatchNormalization(epsilon=bnEps, momentum=bnMom)(c3)
    a3 = Activation('relu')(b3)

    c4 = conv1 = Conv2D(128, (3,3), padding='same', use_bias=False,kernel_initializer='glorot_uniform')(a3)
    b4 = BatchNormalization(epsilon=bnEps, momentum=bnMom)(c4)

    m1  = add([c1, b4])
    a4 = Activation('relu')(m1)

    #----------------------------------------------layer 3------------------------------------------------------------------------------

    c1 = Conv2D(128, (3,3),strides = (1,2) ,padding='same',activation='relu', use_bias=False,kernel_initializer='glorot_uniform')(a4)
    b1 = BatchNormalization(epsilon=bnEps, momentum=bnMom)(c1)

    c2 = conv1 = Conv2D(128, (3,3),strides = (1,2), padding='same', use_bias=False,kernel_initializer='glorot_uniform')(a4)
    b2 = BatchNormalization(epsilon=bnEps, momentum=bnMom)(c2)
    a2 = Activation('relu')(b2)

    c3 = conv1 = Conv2D(128, (3,3), padding='same', use_bias=False,kernel_initializer='glorot_uniform')(a2)
    b3 = BatchNormalization(epsilon=bnEps, momentum=bnMom)(c3)
    a3 = Activation('relu')(b3)


    c4 = conv1 = Conv2D(128, (3,3), padding='same', use_bias=False,kernel_initializer='glorot_uniform')(a3)
    b4 = BatchNormalization(epsilon=bnEps, momentum=bnMom)(c4)

    m1  = add([c1, b4])
    a4 = Activation('relu')(m1)

    #-------------------------------------------layer 4---------------------------------------------------------------------------------

    c1 = Conv2D(64, (3,3),strides = (2,2) ,padding='same',activation='relu', use_bias=False,kernel_initializer='glorot_uniform')(a4)
    b1 = BatchNormalization(epsilon=bnEps, momentum=bnMom)(c1)

    c2 = conv1 = Conv2D(128, (3,3),strides = (2,2), padding='same', use_bias=False,kernel_initializer='glorot_uniform')(a4)
    b2 = BatchNormalization(epsilon=bnEps, momentum=bnMom)(c2)
    a2 = Activation('relu')(b2)

    c3 = conv1 = Conv2D(64, (3,3), padding='same', use_bias=False,kernel_initializer='glorot_uniform')(a2)
    b3 = BatchNormalization(epsilon=bnEps, momentum=bnMom)(c3)
    a3 = Activation('relu')(b3)


    c4 = conv1 = Conv2D(64, (3,3), padding='same', use_bias=False,kernel_initializer='glorot_uniform')(a3)
    b4 = BatchNormalization(epsilon=bnEps, momentum=bnMom)(c4)

    m1  = add([c1, b4])
    a4 = Activation('relu')(m1)

    #-------------------------------------------layer 5-----------------------------------------------------------------------------------
    c1 = Conv2D(32, (3,3),strides = (2,2) ,padding='same',activation='relu', use_bias=False,kernel_initializer='glorot_uniform')(a4)
    b1 = BatchNormalization(epsilon=bnEps, momentum=bnMom)(c1)

    c2 = conv1 = Conv2D(64, (3,3),strides = (2,2), padding='same', use_bias=False,kernel_initializer='glorot_uniform')(a4)
    b2 = BatchNormalization(epsilon=bnEps, momentum=bnMom)(c2)
    a2 = Activation('relu')(b2)

    c3 = conv1 = Conv2D(64, (3,3), padding='same', use_bias=False,kernel_initializer='glorot_uniform')(a2)
    b3 = BatchNormalization(epsilon=bnEps, momentum=bnMom)(c3)
    a3 = Activation('relu')(b3)


    c4 = conv1 = Conv2D(32, (3,3), padding='same', use_bias=False,kernel_initializer='glorot_uniform')(a3)
    b4 = BatchNormalization(epsilon=bnEps, momentum=bnMom)(c4)

    m1  = add([c1, b4])
    a4 = Activation('relu')(m1)

    #-----------------------------------------layer 6-------------------------------------------------------------------------
    c1 = Conv2D(16, (3,3),strides = (2,2) ,padding='same',activation='relu', use_bias=False,kernel_initializer='glorot_uniform')(a4)
    b1 = BatchNormalization(epsilon=bnEps, momentum=bnMom)(c1)

    c2 = conv1 = Conv2D(32, (3,3),strides = (2,2), padding='same', use_bias=False,kernel_initializer='glorot_uniform')(a4)
    b2 = BatchNormalization(epsilon=bnEps, momentum=bnMom)(c2)
    a2 = Activation('relu')(b2)

    c3 = conv1 = Conv2D(32, (3,3), padding='same', use_bias=False,kernel_initializer='glorot_uniform')(a2)
    b3 = BatchNormalization(epsilon=bnEps, momentum=bnMom)(c3)
    a3 = Activation('relu')(b3)


    c4 = conv1 = Conv2D(16, (3,3), padding='same',use_bias=False,kernel_initializer='glorot_uniform')(a3)
    b4 = BatchNormalization(epsilon=bnEps, momentum=bnMom)(c4)

    m1  = add([c1, b4])
    a4 = Activation('relu')(m1)

    f = Flatten()(a4)
    # f = Reshape((int(8192/2), 1))(f)

    # #-----------------------------------------layer7---------------------------------------------------------------------------
    # bi1 = Bidirectional(CuDNNLSTM(512, return_sequences=True))(f)
    bi1 = Dense(1024,activation='relu')(f)
    d1  = Dropout(0.2)(bi1)

    # bi2 = Bidirectional(CuDNNLSTM(512))(d1)
    bi2 = Dense(512,activation='relu')(d1)
    d2 = Dropout(0.4)(bi2)

    out = Dense(2,activation='sigmoid')(d2)

    # create model
    model = Model(inputs=input, outputs=out)
    return model

In [10]:
#both repetitions and fillers
fandr = data[data['filler'] == 1]
fandr = fandr[fandr['repetition'] == 1]

#only fillers
f = data[data['repetition'] == 0]
f = f[f['filler'] == 1]

#only repetitions
r = data[data['filler'] == 0]
r = r[r['repetition'] == 1]

#only perfect sentences
perfect = data[data['filler'] == 0]
perfect = perfect[perfect['repetition'] == 0]

fandr.shape,f.shape,r.shape,perfect.shape

((105, 3), (1219, 3), (640, 3), (10293, 3))

In [11]:
def splitting(df):
  msk = np.random.rand(len(df)) < 0.8
  train = df[msk]
  test = df[~msk]
  return train,test

train_fandr,test_fandr = splitting(fandr)
train_f    ,test_f     = splitting(f)
train_r    ,test_r     = splitting(r)
train_perfect,test_perfect = splitting(perfect)

train_fandr,val_fandr = splitting(train_fandr)
train_f    ,val_f     = splitting(train_f)
train_r    ,val_r     = splitting(train_r)
train_perfect,val_perfect = splitting(train_perfect)


print(train_fandr.shape,val_fandr.shape,test_fandr.shape)
print(train_f.shape,val_r.shape,test_f.shape)
print(train_r.shape,val_r.shape,test_r.shape)
print(train_perfect.shape,val_perfect.shape,test_perfect.shape)


(60, 3) (19, 3) (26, 3)
(783, 3) (97, 3) (245, 3)
(410, 3) (97, 3) (133, 3)
(6647, 3) (1659, 3) (1987, 3)


In [13]:
def get_data(perfect,f,r,fandr,batch=8,size = (256*8,256),root_path = '/content/spectrograms/'):
    
     

    while True:
        imp_data = []
        count = 0

        def read_image(file_name):
            img = cv2.imread(file_name,0)
            img =cv2.resize(img,size)
            img = np.expand_dims(img, axis=-1)
            return img

        perfect = perfect.sample(frac=1)
        f = f.sample(frac=1)
        r = r.sample(frac =1)
        fandr = fandr.sample(frac = 1)

        # print(fandr.head(5))

        for i in np.array(fandr.sample(frac=1).head(batch//4)):
            image = read_image(root_path+i[0])
            labels = list(i[1:])
            imp_data.append([image,labels])
            # print('fandr')
        
        for i in np.array(r.sample(frac=1).head(batch//4)):
            image = read_image(root_path+i[0])
            labels = list(i[1:])
            imp_data.append([image,labels])
            # print('f')


        for i in np.array(f.sample(frac=1).head(batch//4)):
            image = read_image(root_path+i[0])
            labels = list(i[1:])
            imp_data.append([image,labels])
            # print('r')


        for i in np.array(perfect.sample(frac=1).head(batch//4)):
            image = read_image(root_path+i[0])
            labels = list(i[1:])
            imp_data.append([image,labels])
            # print('perfect')

        imp_data = shuffle(imp_data)
        # imp_data = np.array(imp_data)

        # print(imp_data.shape)
        
        images = [i[0] for i in imp_data]
        # print(np.array(images).shape)
        labels = [i[1] for i in imp_data]

        yield (np.array(images)/255,np.array(labels))

In [15]:
from keras import backend



# calculate fbeta score for multi-class/label classification
def fbeta(y_true, y_pred, beta=2):
	# clip predictions
	y_pred = backend.clip(y_pred, 0, 1)
	# calculate elements
	tp = backend.sum(backend.round(backend.clip(y_true * y_pred, 0, 1)), axis=1)
	fp = backend.sum(backend.round(backend.clip(y_pred - y_true, 0, 1)), axis=1)
	fn = backend.sum(backend.round(backend.clip(y_true - y_pred, 0, 1)), axis=1)
	# calculate precision
	p = tp / (tp + fp + backend.epsilon())
	# calculate recall
	r = tp / (tp + fn + backend.epsilon())
	# calculate fbeta, averaged across each class
	bb = beta ** 2
	fbeta_score = backend.mean((1 + bb) * (p * r) / (bb * p + r + backend.epsilon()))
	return fbeta_score

In [17]:
w = 2048

#https://machinelearningmastery.com/how-to-develop-a-convolutional-neural-network-to-classify-satellite-photos-of-the-amazon-rainforest/

model = resnet_model_dilation(size = (256,w,1))

model.compile(loss='binary_crossentropy',optimizer=optimizers.SGD(lr=1e-4,momentum=0.5),metrics=[fbeta])

my_callbacks = [
    keras.callbacks.EarlyStopping(patience=5, monitor="val_loss",min_delta=0.00001,verbose = 1),
    keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5',verbose = 1,save_best_only=True)
]

# score = fbeta_score(y_true, y_pred, 2, average='samples')

batch_size = 16

history = model.fit_generator(get_data(train_perfect,train_f,train_r,train_fandr,size=(w,256)),
                              steps_per_epoch=int((72+757+423+500)/batch_size),               #sum of all the train samples can be checked from above shapes
                              epochs=30,
                              validation_data=get_data(val_perfect,val_f,val_r,val_fandr,size=(w,256)),
                              validation_steps=int((14+108+108+100)/batch_size),              #sum of all the val samples can be checked from above shapes
                              callbacks = my_callbacks)

Epoch 1/30
109/109 [==============================] - 144s 1s/step - loss: 0.8321 - fbeta: 0.3858 - val_loss: 0.7766 - val_fbeta: 0.4326

Epoch 00001: val_loss improved from inf to 0.77665, saving model to model.01-0.78.h5
Epoch 2/30
109/109 [==============================] - 141s 1s/step - loss: 0.6780 - fbeta: 0.4154 - val_loss: 0.7604 - val_fbeta: 0.0962

Epoch 00002: val_loss improved from 0.77665 to 0.76040, saving model to model.02-0.76.h5
Epoch 3/30
109/109 [==============================] - 140s 1s/step - loss: 0.5934 - fbeta: 0.4665 - val_loss: 0.7496 - val_fbeta: 0.3684

Epoch 00003: val_loss improved from 0.76040 to 0.74960, saving model to model.03-0.75.h5
Epoch 4/30
109/109 [==============================] - 140s 1s/step - loss: 0.5635 - fbeta: 0.4927 - val_loss: 0.8893 - val_fbeta: 0.2628

Epoch 00004: val_loss did not improve from 0.74960
Epoch 5/30
109/109 [==============================] - 140s 1s/step - loss: 0.5179 - fbeta: 0.5075 - val_loss: 0.6584 - val_fbeta: 0.32

In [18]:
w = 2048

#https://machinelearningmastery.com/how-to-develop-a-convolutional-neural-network-to-classify-satellite-photos-of-the-amazon-rainforest/

model = resnet_model_dilation(size = (256,w,1))

model.compile(loss='binary_crossentropy',optimizer=optimizers.SGD(lr=1e-4,momentum=0.5),metrics=[fbeta])

my_callbacks = [
    keras.callbacks.EarlyStopping(patience=5, monitor="val_loss",min_delta=0.00001,verbose = 1),
    keras.callbacks.ModelCheckpoint(filepath='model.{epoch:02d}-{val_loss:.2f}.h5',verbose = 1,save_best_only=True)
]

# score = fbeta_score(y_true, y_pred, 2, average='samples')

batch_size = 16

history = model.fit_generator(get_data(train_perfect,train_f,train_r,train_fandr,size=(w,256)),
                              steps_per_epoch=int((72+757+423+500)/batch_size),               #sum of all the train samples can be checked from above shapes
                              epochs=30,
                              validation_data=get_data(val_perfect,val_f,val_r,val_fandr,size=(w,256)),
                              validation_steps=int((14+108+108+100)/batch_size),              #sum of all the val samples can be checked from above shapes
                              callbacks = my_callbacks)

Epoch 1/30
109/109 [==============================] - 142s 1s/step - loss: 0.8640 - fbeta: 0.3821 - val_loss: 0.5995 - val_fbeta: 0.3726

Epoch 00001: val_loss improved from inf to 0.59948, saving model to model.01-0.60.h5
Epoch 2/30
109/109 [==============================] - 139s 1s/step - loss: 0.8653 - fbeta: 0.3586 - val_loss: 0.7837 - val_fbeta: 0.3306

Epoch 00002: val_loss did not improve from 0.59948
Epoch 3/30
109/109 [==============================] - 138s 1s/step - loss: 0.8239 - fbeta: 0.3883 - val_loss: 0.7591 - val_fbeta: 0.3187

Epoch 00003: val_loss did not improve from 0.59948
Epoch 4/30
109/109 [==============================] - 138s 1s/step - loss: 0.7905 - fbeta: 0.3829 - val_loss: 0.6580 - val_fbeta: 0.2722

Epoch 00004: val_loss did not improve from 0.59948
Epoch 5/30
109/109 [==============================] - 138s 1s/step - loss: 0.7803 - fbeta: 0.3682 - val_loss: 0.6032 - val_fbeta: 0.3764

Epoch 00005: val_loss did not improve from 0.59948
Epoch 6/30
109/109 [=